In [1]:
import re
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

In [2]:
df = pd.read_csv("./IMDB Dataset.csv")

In [3]:
features = df.sample(len(df)).reindex(index=range(len(df))).copy()
label = features.pop('sentiment')

In [4]:
SYMBOL_FILTER = re.compile("[!,.\"\':?()]")
SPECIAL_CHAR_FILTER = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

In [5]:
features['review'] = features.review.apply(lambda x:SPECIAL_CHAR_FILTER.sub(
    "",
    SYMBOL_FILTER.sub(
        "",
        x
    )
).lower())

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
count = CountVectorizer(dtype=np.uint8)

In [8]:
vect = count.fit(features['review'].values)

In [9]:
vects = vect.transform(features['review'].values)

In [10]:
labels = (label.values == 'positive').astype(np.uint8)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X,x,Y,y = train_test_split(vects,labels,random_state = 42)

In [13]:
from sklearn.metrics import classification_report

In [14]:
from tensorflow import keras
from tensorflow.keras.layers import *

In [19]:
model = keras.Sequential([
    Dense(512,activation="relu",input_shape=(X.shape[1],)),
    Dense(256,activation="relu"),
    Dense(128,activation="relu"),
    Dense(1,activation="sigmoid")
])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               113052672 
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 113,217,025
Trainable params: 113,217,025
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer="adam",loss="binary_crossentropy")

In [51]:
def get_flow(x,y,epochs,batch_size):
    br = x.shape[0]%batch_size
    batches = np.arange(0,x.shape[0]-br,batch_size)
    
    x_ = x[-br:]
    y_ = y[-br:]

    def flow():
        for na in range(epochs):
            for batch in batches:
                yield x[batch:batch+batch_size].toarray(),y[batch:batch+batch_size]            
        yield x_.toarray(),y_
        
    return flow(),len(batches)+1

In [52]:
epochs = 3
batch_size = 128

train_flow,steps = get_flow(X,Y,epochs,batch_size)

In [53]:
model.fit_generator(train_flow,steps_per_epoch=steps,epochs=epochs)

In [54]:
epochs = 1
batch_size = 32

test_flow,steps = get_flow(x,y,epochs,batch_size)

In [56]:
model.evaluate_generator(test_flow,steps)

  ...
    to  
  ['...']


0.4563531208702404

In [64]:
epochs = 1
batch_size = 32

test_flow,steps = get_flow(x,y,epochs,batch_size)
pred = model.predict_generator(test_flow,steps)

pred = (pred > 0.9).astype(np.uint8).reshape(-1)
print (classification_report(y,pred))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89      6157
           1       0.93      0.83      0.88      6343

    accuracy                           0.88     12500
   macro avg       0.89      0.88      0.88     12500
weighted avg       0.89      0.88      0.88     12500



In [66]:
epochs = 1
batch_size = 32

test_flow,steps = get_flow(X,Y,epochs,batch_size)
pred = model.predict_generator(test_flow,steps)

pred = (pred > 0.9).astype(np.uint8).reshape(-1)
print (classification_report(Y,pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     18843
           1       1.00      0.99      1.00     18657

    accuracy                           1.00     37500
   macro avg       1.00      1.00      1.00     37500
weighted avg       1.00      1.00      1.00     37500



In [71]:
sent = "I liked movie"
model.predict(vect.transform([sent]).toarray())

array([[0.9868765]], dtype=float32)

In [72]:
sent = "I didnt like the movie"
model.predict(vect.transform([sent]).toarray())

array([[0.061299]], dtype=float32)